<a href="https://colab.research.google.com/github/kzing20/ML/blob/main/%EA%B0%90%EC%A0%95%EC%9D%B8%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리 설치

In [ ]:
!pip install transformers==4.4.0
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 40.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 49.8 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=890fda3fa0db56f4938a87a32012084494854c6d24951211d203c9101bd0ba6e
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=796f561acfb88ea7df9bca2168ef0514c9392cb423ef0f0f7d25317605ddb817
  Stored in directory: /root/

In [ ]:
!nvidia-smi

Thu Jan 19 12:24:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.__version__

'1.13.1+cu116'

In [ ]:
import transformers
transformers.__version__

'4.4.0'

# 데이터 다운로드

In [ ]:
!git clone https://github.com/declare-lab/MELD.git

Cloning into 'MELD'...
remote: Enumerating objects: 487, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 487 (delta 6), reused 0 (delta 0), pack-reused 475
Receiving objects: 100% (487/487), 8.12 MiB | 19.51 MiB/s, done.
Resolving deltas: 100% (254/254), done.


In [ ]:
import glob
data_path="./MELD/data/MELD/*.csv"
data_path_list = glob.glob(data_path)
print(data_path_list)

['./MELD/data/MELD/train_sent_emo.csv', './MELD/data/MELD/dev_sent_emo.csv', './MELD/data/MELD/test_sent_emo.csv']


# 데이터 확인

In [ ]:
!head -5 './MELD/data/MELD/dev_sent_emo.csv'

Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
1,"Oh my God, he’s lost it. He’s totally lost it.",Phoebe,sadness,negative,0,0,4,7,"00:20:57,256","00:21:00,049"
2,What?,Monica,surprise,negative,0,1,4,7,"00:21:01,927","00:21:03,261"
3,"Or! Or, we could go to the bank, close our accounts and cut them off at the source.",Ross,neutral,neutral,1,0,4,4,"00:12:24,660","00:12:30,915"
4,You’re a genius!,Chandler,joy,positive,1,1,4,4,"00:12:32,334","00:12:33,960"


In [ ]:
#데이터 출력
import csv
for data_path in data_path_list:
  f=open(data_path,'r')
  rdr = csv.reader(f)

  for line in rdr:
    print(line)
    break

  f.close()
  break

['Sr No.', 'Utterance', 'Speaker', 'Emotion', 'Sentiment', 'Dialogue_ID', 'Utterance_ID', 'Season', 'Episode', 'StartTime', 'EndTime']


# 세션으로 데이터 분할하기

In [ ]:
#데이터 저장
def split(session):
  final_data = []
  split_session = []
  for line in session:
    split_session.append(line)
    final_data.append(split_session[:])
  return final_data

# 유니크한 스피커로 바꾸기
def uniq(speaker_set):
  if speaker in speaker_set:
    uniq_speaker = speaker_set.index(speaker)
  else:
    speaker_set.append(speaker)
    uniq_speaker= speaker_set.index(speaker) 
  return uniq_speaker

for data_path in data_path_list:
  f = open(data_path,'r')
  rdr = csv.reader(f)

  """ 세션 데이터 저장할 것 """
  session_dataset = []
  session = []
  speaker_set =[]

  """ 실제 데이터 저장 방식 """
  pre_sess = 'start'
  for i, line in enumerate(rdr):
    if i==0:
      """저장할 데이터들 index 확인"""
      header = line
      utt_idx= header.index('Utterance')
      speaker_idx = header.index('Speaker')
      emo_idx = header.index('Emotion')
      sess_idx = header.index('Dialogue_ID')
    else:
      utt = line[utt_idx]
      speaker = line[speaker_idx]
      emotion = line[emo_idx]
      sess = line[sess_idx]
      if pre_sess == 'start' or sess == pre_sess:
        uniq_speaker = uniq(speaker_set)  
        session.append([uniq_speaker,utt,emotion])
      else:
        """세션 데이터 저장"""
        session_dataset += split(session)
        speaker_set = []
        uniq_speaker = uniq(speaker_set)   
        session = [[uniq_speaker,utt,emotion]]
    
      pre_sess = sess

  """마지막 세션 저장"""
  session_dataset += split(session)
  f.close()

  break

In [ ]:
a= [1,2,3]
a = split(a)
a

[[1], [1, 2], [1, 2, 3]]

In [ ]:
a[0]

[1]

In [ ]:
session_dataset[0]

[[0,
  'also I was the point person on my company’s transition from the KL-5 to GR-6 system.',
  'neutral']]

In [ ]:
session_dataset[1]

[[0,
  'also I was the point person on my company’s transition from the KL-5 to GR-6 system.',
  'neutral'],
 [1, 'You must’ve had your hands full.', 'neutral']]

In [ ]:
session_dataset[2]

[[0,
  'also I was the point person on my company’s transition from the KL-5 to GR-6 system.',
  'neutral'],
 [1, 'You must’ve had your hands full.', 'neutral'],
 [0, 'That I did. That I did.', 'neutral']]

In [ ]:
session_dataset[3]

[[0,
  'also I was the point person on my company’s transition from the KL-5 to GR-6 system.',
  'neutral'],
 [1, 'You must’ve had your hands full.', 'neutral'],
 [0, 'That I did. That I did.', 'neutral'],
 [1, 'So let’s talk a little bit about your duties.', 'neutral']]

In [ ]:
session_dataset[4]

[[0,
  'also I was the point person on my company’s transition from the KL-5 to GR-6 system.',
  'neutral'],
 [1, 'You must’ve had your hands full.', 'neutral'],
 [0, 'That I did. That I did.', 'neutral'],
 [1, 'So let’s talk a little bit about your duties.', 'neutral'],
 [0, 'My duties?  All right.', 'surprise']]

In [ ]:
session_dataset[10]

[[0,
  'also I was the point person on my company’s transition from the KL-5 to GR-6 system.',
  'neutral'],
 [1, 'You must’ve had your hands full.', 'neutral'],
 [0, 'That I did. That I did.', 'neutral'],
 [1, 'So let’s talk a little bit about your duties.', 'neutral'],
 [0, 'My duties?  All right.', 'surprise'],
 [1,
  'Now you’ll be heading a whole division, so you’ll have a lot of duties.',
  'neutral'],
 [0, 'I see.', 'neutral'],
 [1,
  'But there’ll be perhaps 30 people under you so you can dump a certain amount on them.',
  'neutral'],
 [0, 'Good to know.', 'neutral'],
 [1, 'We can go into detail', 'neutral'],
 [0, 'No don’t I beg of you!', 'fear']]

In [ ]:
session_dataset[28]

[[0, 'Hey, Mon.', 'neutral'],
 [1, 'Hey-hey-hey. You wanna hear something that sucks.', 'neutral'],
 [0, 'Do I ever.', 'joy'],
 [1, 'Chris says they’re closing down the bar.', 'sadness'],
 [0, 'No way!', 'surprise'],
 [1,
  'Yeah, apparently they’re turning it into some kinda coffee place.',
  'neutral'],
 [0, 'Just coffee! Where are we gonna hang out now?', 'disgust'],
 [1, 'Got me.', 'sadness']]

## 실제 감정인식에 맞게 데이터 분할하기 (배치처리 알아보기)

In [ ]:
import csv
from torch.utils.data import Dataset
def split(session):
    final_data = []
    split_session = []
    for line in session:
        split_session.append(line)
        final_data.append(split_session[:])    
    return final_data

class data_loader(Dataset):
    def __init__(self, data_path):
        f = open(data_path, 'r')
        rdr = csv.reader(f)
        
        """ 추가 """
        emoSet = set()

        """ 세션 데이터 저장할 것"""
        self.session_dataset = []
        session = []
        speaker_set = []

        """ 실제 데이터 저장 방식 """
        pre_sess = 'start'
        for i, line in enumerate(rdr):
            if i == 0:
                """ 저장할 데이터들 index 확인 """
                header  = line
                utt_idx = header.index('Utterance')
                speaker_idx = header.index('Speaker')
                emo_idx = header.index('Emotion')
                sess_idx = header.index('Dialogue_ID')
            else:
                utt = line[utt_idx]
                speaker = line[speaker_idx]
                """ 유니크한 스피커로 바꾸기 """
                if speaker in speaker_set:
                    uniq_speaker = speaker_set.index(speaker)
                else:
                    speaker_set.append(speaker)
                    uniq_speaker = speaker_set.index(speaker)
                emotion = line[emo_idx]
                sess = line[sess_idx]

                if pre_sess == 'start' or sess == pre_sess:
                    session.append([uniq_speaker, utt, emotion])
                else:
                    """ 세션 데이터 저장 """
                    self.session_dataset += split(session)
                    session = [[uniq_speaker, utt, emotion]]
                    speaker_set = []
                    emoSet.add(emotion)
                pre_sess = sess   
        """ 마지막 세션 저장 """
        self.session_dataset += split(session)
            
        """ 추가 """
        # self.emoList = sorted(emoSet) # 항상 같은 레이블 순서를 유지하기 위해
        self.emoList = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
        f.close()
        
    def __len__(self): # 기본적인 구성
        return len(self.session_dataset)
    
    def __getitem__(self, idx): # 기본적인 구성
        return self.session_dataset[idx]
    
    def collate_fn(self, sessions): # 배치를 위한 구성
        '''
            input:
                data: [(session1), (session2), ... ]
            return:
                batch_input_tokens_pad: (B, L) padded
                batch_labels: (B)
        '''
        batch_input_token = []
        for session in sessions:
            input_token = ""
            for line in session:
                speaker, utt, emotion = line
                input_token += utt
            batch_input_token.append(input_token)
        
        return batch_input_token

In [ ]:
dev_dataset = data_loader('./MELD/data/MELD/dev_sent_emo.csv')
dev_dataset[0]

[[0, 'Oh my God, he’s lost it. He’s totally lost it.', 'sadness']]

In [ ]:
dev_dataset[1]

[[0, 'Oh my God, he’s lost it. He’s totally lost it.', 'sadness'],
 [1, 'What?', 'surprise']]

In [ ]:
""" 배치 결과 확인 """
from torch.utils.data import DataLoader
dev_dataloader = DataLoader(dev_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=dev_dataset.collate_fn)

i = 0
for data in dev_dataloader:
    print(i, data)
    i += 1
    if i > 2:
        break    

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


0 ['Oh my God, he’s lost it. He’s totally lost it.']
1 ['Oh my God, he’s lost it. He’s totally lost it.What?']
2 ['Or! Or, we could go to the bank, close our accounts and cut them off at the source.']


In [ ]:
""" 배치 결과 확인 """
from torch.utils.data import DataLoader
dev_dataloader = DataLoader(dev_dataset, batch_size=3, shuffle=False, num_workers=4, collate_fn=dev_dataset.collate_fn)
i = 0
for data in dev_dataloader:
    print(i,data)
    i += 1
    if i > 2:
      break

0 ['Oh my God, he’s lost it. He’s totally lost it.', 'Oh my God, he’s lost it. He’s totally lost it.What?', 'Or! Or, we could go to the bank, close our accounts and cut them off at the source.']
1 ['Or! Or, we could go to the bank, close our accounts and cut them off at the source.You’re a genius!', 'Or! Or, we could go to the bank, close our accounts and cut them off at the source.You’re a genius!Aww, man, now we won’t be bank buddies!', 'Or! Or, we could go to the bank, close our accounts and cut them off at the source.You’re a genius!Aww, man, now we won’t be bank buddies!Now, there’s two reasons.']
2 ['Or! Or, we could go to the bank, close our accounts and cut them off at the source.You’re a genius!Aww, man, now we won’t be bank buddies!Now, there’s two reasons.Hey.', 'Or! Or, we could go to the bank, close our accounts and cut them off at the source.You’re a genius!Aww, man, now we won’t be bank buddies!Now, there’s two reasons.Hey.Hey!', 'Or! Or, we could go to the bank, close o

## 사전 학습 모델들에 대한 백그라운드

In [ ]:
""" 토크나이저 확인하기 """
# https://github.com/thunlp/PLMpapers
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
print(tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token)
print(tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id)

<s> </s> <pad>
0 2 1


In [ ]:
#dir(tokenizer)
tokenizer.model_max_length

512

In [ ]:
""" 토크나이저 작동 """
res = tokenizer('hello. this is fastcampus')
print(res)
res = tokenizer.encode('hello. this is fastcampus')
print(res)
res = tokenizer(['hello. this is fastcampus', "what are you doing?"])
print(res)
res = tokenizer(['hello. this is fastcampus', "what are you doing?"], add_special_tokens=False)
print(res)

{'input_ids': [0, 42891, 4, 42, 16, 1769, 28135, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
[0, 42891, 4, 42, 16, 1769, 28135, 2]
{'input_ids': [[0, 42891, 4, 42, 16, 1769, 28135, 2], [0, 12196, 32, 47, 608, 116, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}
{'input_ids': [[42891, 4, 42, 16, 1769, 28135], [12196, 32, 47, 608, 116]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}


In [ ]:
""" 배치 입력 토큰들 처리 """
from torch.utils.data import Dataset
from transformers import RobertaTokenizer
import csv
from torch.utils.data import Dataset
import torch

def split(session):
    final_data = []
    split_session = []
    for line in session:
        split_session.append(line)
        final_data.append(split_session[:])    
    return final_data

class data_loader(Dataset):
    def __init__(self, data_path):
        f = open(data_path, 'r')
        rdr = csv.reader(f)
        
        """ 추가 """
        emoSet = set()
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

        """ 세션 데이터 저장할 것"""
        self.session_dataset = []
        session = []
        speaker_set = []

        """ 실제 데이터 저장 방식 """
        pre_sess = 'start'
        for i, line in enumerate(rdr):
            if i == 0:
                """ 저장할 데이터들 index 확인 """
                header  = line
                utt_idx = header.index('Utterance')
                speaker_idx = header.index('Speaker')
                emo_idx = header.index('Emotion')
                sess_idx = header.index('Dialogue_ID')
            else:
                utt = line[utt_idx]
                speaker = line[speaker_idx]
                """ 유니크한 스피커로 바꾸기 """
                if speaker in speaker_set:
                    uniq_speaker = speaker_set.index(speaker)
                else:
                    speaker_set.append(speaker)
                    uniq_speaker = speaker_set.index(speaker)
                emotion = line[emo_idx]
                sess = line[sess_idx]

                if pre_sess == 'start' or sess == pre_sess:
                    session.append([uniq_speaker, utt, emotion])
                else:
                    """ 세션 데이터 저장 """
                    self.session_dataset += split(session)
                    session = [[uniq_speaker, utt, emotion]]
                    speaker_set = []
                    emoSet.add(emotion)
                pre_sess = sess   
        """ 마지막 세션 저장 """
        self.session_dataset += split(session)
            
        # self.emoList = sorted(emoSet) # 항상 같은 레이블 순서를 유지하기 위해
        self.emoList = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
        f.close()
        
    def __len__(self): # 기본적인 구성
        return len(self.session_dataset)
    
    def __getitem__(self, idx): # 기본적인 구성
        return self.session_dataset[idx]
    
    def padding(self, batch_input_token):
        """ 추가 """
        """ 512 토큰 길이 넘으면 잘라내기 """
        batch_token_ids, batch_attention_masks = batch_input_token['input_ids'], batch_input_token['attention_mask']
        trunc_batch_token_ids, trunc_batch_attention_masks = [], []
        for batch_token_id, batch_attention_mask in zip(batch_token_ids, batch_attention_masks):
            if len(batch_token_id) > self.tokenizer.model_max_length:
                trunc_batch_token_id = [batch_token_id[0]] + batch_token_id[1:][-self.tokenizer.model_max_length+1:]
                trunc_batch_attention_mask = [batch_attention_mask[0]] + batch_attention_mask[1:][-self.tokenizer.model_max_length+1:]
                trunc_batch_token_ids.append(trunc_batch_token_id)
                trunc_batch_attention_masks.append(trunc_batch_attention_mask)
            else:
                trunc_batch_token_ids.append(batch_token_id)
                trunc_batch_attention_masks.append(batch_attention_mask)
        
        """ padding token으로 패딩하기 """
        # [10, 30, 50]
        # [50, 50, 50] 
        # 50-10=40 , 50-30=20 : 패딩토큰으로 채운다. <pad>
        max_length = max([len(x) for x in trunc_batch_token_ids])
        padding_tokens, padding_attention_masks = [], []
        for batch_token_id, batch_attention_mask in zip(trunc_batch_token_ids, trunc_batch_attention_masks):
            padding_tokens.append(batch_token_id + [self.tokenizer.pad_token_id for _ in range(max_length-len(batch_token_id))])
            padding_attention_masks.append(batch_attention_mask + [0 for _ in range(max_length-len(batch_token_id))]                                                        )
        return torch.tensor(padding_tokens), torch.tensor(padding_attention_masks)
    
    def collate_fn(self, sessions): # 배치를 위한 구성
        '''
            input:
                data: [(session1), (session2), ... ]
            return:
                batch_input_tokens_pad: (B, L) padded
                batch_labels: (B)
        '''
        ## [발화1, 발화2, ..., 발화8]
        # 발화1~발화7 컨텍스트로 사용한다면 입력이 길어진다.
        # 발화1 같은 경우는 발화8에 덜중요할거에요.
        # 적절하게 컨텍스트 길이를 조절해도된다.
        # 3개로 정한다면, [발화5,발화6,발화7,발화8]
        """ 추가 """
        batch_input, batch_labels = [], []
        batch_PM_input = []
        for session in sessions:
            input_str = self.tokenizer.cls_token
            
            """ For PM """
            current_speaker, current_utt, current_emotion = session[-1]
            PM_input = []
            for i, line in enumerate(session):
                speaker, utt, emotion = line
                input_str += " " + utt + self.tokenizer.sep_token
                if i < len(session)-1 and current_speaker == speaker:
                    PM_input.append(self.tokenizer.encode(utt, add_special_tokens=True, return_tensors='pt'))
                    # [cls_token, tokens, sep_token]
                    
            """ For CoM """
            batch_input.append(input_str)
            batch_labels.append(self.emoList.index(emotion))
            batch_PM_input.append(PM_input)
        batch_input_token = self.tokenizer(batch_input, add_special_tokens=False)
        batch_padding_token, batch_padding_attention_mask = self.padding(batch_input_token)
        
        return batch_padding_token, batch_padding_attention_mask, batch_PM_input, torch.tensor(batch_labels)

In [ ]:
""" 배치 결과 확인 """
from torch.utils.data import DataLoader
dev_dataset = data_loader('./MELD/data/MELD/dev_sent_emo.csv')
dev_dataloader = DataLoader(dev_dataset, batch_size=3, shuffle=False, num_workers=4, collate_fn=dev_dataset.collate_fn)

for i, data in enumerate(dev_dataloader):
    if i == 1:
        print(i,data[0].shape)
        batch_padding_token, batch_padding_attention_mask, batch_PM_input, batch_label = data
        print("batch_padding_token", batch_padding_token)
        print("batch_padding_attention_mask", batch_padding_attention_mask)
        print("batch_PM_input", batch_PM_input)
        print("batch_label", batch_label)
        break

1 torch.Size([3, 58])
batch_padding_token tensor([[    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
           827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
             5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
           328,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
           827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
             5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
           328,     2,    83, 33130,     6,   313,     6,   122,    52,   351,
            17,    27,    90,    28,   827, 30489,   328,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  1793,   328,  1793,

# 파일로 저장하기

In [ ]:
!touch dataset.py
# 코드 복사하기

In [ ]:
from dataset import data_loader
from torch.utils.data import DataLoader
dev_dataset = data_loader('./MELD/data/MELD/dev_sent_emo.csv')
dev_dataloader = DataLoader(dev_dataset, batch_size=3, shuffle=False, num_workers=4, collate_fn=dev_dataset.collate_fn)

for i, data in enumerate(dev_dataloader):
    if i == 1:
        print(data[0].shape)
        batch_padding_token, batch_padding_attention_mask, batch_PM_input, batch_label = data
        print("batch_padding_token", batch_padding_token)
        print("batch_padding_attention_mask", batch_padding_attention_mask)
        print("batch_PM_input", batch_PM_input)
        print("batch_label", batch_label)
        break

ImportError: ignored

# **사전학습 모델 불러와서 로딩하기**

In [ ]:
from transformers import RobertaModel
model = RobertaModel.from_pretrained('roberta-base')
print('')

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

In [ ]:
# model
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_convert_head_mask_to_5d',
 '_expand_inputs_for_generation',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_decoder_start_token_id',
 '_get_logits_processor',
 '_get_logits_warper',
 '_get_name',
 '_get_pad_token_id',
 '_get_resized_embeddings',
 '_get_resized_lm_head',
 '_get_stopping_criteria',
 '_hook_rss_memory_post_forward',
 '_hook_rss_memory_pre_forward',
 '_init_sequence_length_for_generation',
 '_init_weights',
 '_is_full_backward_hook',
 '_keys_

In [ ]:
model.train() # dropout의 probability을 이용한다
# 학습코드
model.eval() # dropout의 probability을 이용한다 --> dropout 작동X
# 평가코드

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

# 사전학습 모델 사용해보기

In [ ]:
batch_padding_token.shape, batch_padding_attention_mask.shape

(torch.Size([3, 58]), torch.Size([3, 58]))

In [ ]:
batch_padding_token, batch_padding_attention_mask

(tensor([[    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
            827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
              5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
            328,     2,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1],
         [    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
            827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
              5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
            328,     2,    83, 33130,     6,   313,     6,   122,    52,   351,
             17,    27,    90,    28,   827, 30489,   328,     2,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1],
         [    0,  1793,   328,  1793,     6,    52,   115,   213, 

In [ ]:
""" for CoM """
batch_com_out = model(input_ids=batch_padding_token, attention_mask=batch_padding_attention_mask)['last_hidden_state']
print(batch_com_out.shape)
batch_com_final = batch_com_out[:,0,:] # CLS 토큰의 output 가져오기 위해
print(batch_com_final.shape)

torch.Size([3, 58, 768])
torch.Size([3, 768])


In [ ]:
batch_PM_input

[[],
 [],
 [tensor([[    0,  1185,    17,    27,   241,    10, 16333,   328,     2]])]]

In [ ]:
import torch

model2 = RobertaModel.from_pretrained('roberta-base')
# 발화1: feature1 [1, 768]
# 발화3: feature3 [1, 768]
# 발화6에 해당하는 감정을 예측할 때 발화1, 발화3의 정보를 사용할 것
# feature1 + feature3
# (feature1, feature6) 어텐션 weights w1
# (feature3, feature6) 어텐션 weights w3
# w1*feature1 + w3*feature6
# GRU(feature1, feature3)

""" GRU 세팅 """
import torch.nn as nn 
hiddenDim = model2.config.hidden_size
zero = torch.empty(2, 1, hiddenDim)
h0 = torch.zeros_like(zero) # (num_layers * num_directions, batch, hidden_size)
speakerGRU = nn.GRU(hiddenDim, hiddenDim, 2, dropout=0.3) # (input, hidden, num_layer) (BERT_emb, BERT_emb, num_layer)

""" GRU 통과 --> PM 결과 """
batch_pm_gru_final = []
for PM_inputs in batch_PM_input:
    if PM_inputs:
        pm_outs = []
        for PM_input in PM_inputs:
            pm_out = model2(PM_input)['last_hidden_state'][:,0,:] # CLS의 출력
            pm_outs.append(pm_out)
        pm_outs = torch.cat(pm_outs, 0).unsqueeze(1) # (speaker_num, batch=1, hidden_dim)
        pm_gru_outs, _ = speakerGRU(pm_outs, h0) # (speaker_num, batch=1, hidden_dim)
        pm_gru_final = pm_gru_outs[-1,:,:] # (1, hidden_dim)
        batch_pm_gru_final.append(pm_gru_final)
    else:
        batch_pm_gru_final.append(torch.zeros(1, hiddenDim))
batch_pm_gru_final = torch.cat(batch_pm_gru_final, 0)

# 추가적인 layer 구성하기

In [ ]:
""" score matrix """
clsNum = len(dev_dataset.emoList)
W = nn.Linear(hiddenDim, clsNum)
final_output = W(batch_com_final + batch_pm_gru_final)
print(final_output.shape) # (B, C)|

torch.Size([3, 7])


In [ ]:
from transformers import RobertaModel
import torch
import torch.nn as nn

class ERC_model(nn.Module):
    def __init__(self, clsNum):
        super(ERC_model, self).__init__()
        self.com_model = RobertaModel.from_pretrained('roberta-base')
        self.pm_model = RobertaModel.from_pretrained('roberta-base')
        
        """ GRU 세팅 """
        self.hiddenDim = self.com_model.config.hidden_size
        zero = torch.empty(2, 1, self.hiddenDim)
        self.h0 = torch.zeros_like(zero) # (num_layers * num_directions, batch, hidden_size)
        self.speakerGRU = nn.GRU(self.hiddenDim, self.hiddenDim, 2, dropout=0.3) # (input, hidden, num_layer) (BERT_emb, BERT_emb, num_layer)
        
        """ score matrix """
        self.W = nn.Linear(self.hiddenDim, clsNum)
    def forward(self, batch_padding_token, batch_padding_attention_mask, batch_PM_input):
        """ for CoM """
        batch_com_out = self.com_model(input_ids=batch_padding_token, attention_mask=batch_padding_attention_mask)['last_hidden_state']
        batch_com_final = batch_com_out[:,0,:]
        
        """ GRU 통과 --> PM 결과 """
        batch_pm_gru_final = []
        for PM_inputs in batch_PM_input:
            if PM_inputs:
                pm_outs = []
                for PM_input in PM_inputs:
                    pm_out = self.pm_model(PM_input)['last_hidden_state'][:,0,:]
                    pm_outs.append(pm_out)
                pm_outs = torch.cat(pm_outs, 0).unsqueeze(1) # (speaker_num, batch=1, hidden_dim)
                pm_gru_outs, _ = self.speakerGRU(pm_outs, self.h0) # (speaker_num, batch=1, hidden_dim)
                pm_gru_final = pm_gru_outs[-1,:,:] # (1, hidden_dim)
                batch_pm_gru_final.append(pm_gru_final)
            else:
                batch_pm_gru_final.append(torch.zeros(1, self.hiddenDim))
        batch_pm_gru_final = torch.cat(batch_pm_gru_final, 0)        
        
        """ score matrix """
        final_output = self.W(batch_com_final + batch_pm_gru_final) # (B, C)
        
        return final_output

In [ ]:
clsNum = len(dev_dataset.emoList)
model = ERC_model(clsNum)
result = model(batch_padding_token, batch_padding_attention_mask, batch_PM_input)
print(result.shape)

torch.Size([3, 7])


# 파일로 저장하기

In [ ]:
!touch model.py


# **학습 코드 짜기**

## 평가 설명하기

In [ ]:
import torch
from tqdm import tqdm
def CalACC(model, dataloader):
    model.eval()
    correct = 0
    label_list = []
    pred_list = []
    
    # label arragne
    with torch.no_grad():
        for i_batch, data in enumerate(tqdm(dataloader)):
            """Prediction"""
            batch_padding_token, batch_padding_attention_mask, batch_PM_input, batch_label = data
            batch_padding_token = batch_padding_token.cuda()
            batch_padding_attention_mask = batch_padding_attention_mask.cuda()
            batch_PM_input = [[x2.cuda() for x2 in x1] for x1 in batch_PM_input]
            batch_label = batch_label.cuda()        

            """Prediction"""
            pred_logits = erc_model(batch_padding_token, batch_padding_attention_mask, batch_PM_input)
            
            """Calculation"""    
            pred_label = pred_logits.argmax(1).item()
            true_label = batch_label.item()
            
            pred_list.append(pred_label)
            label_list.append(true_label)
            if pred_label == true_label:
                correct += 1
        acc = correct/len(dataloader)
    return acc, pred_list, label_list

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
dev_acc, dev_pred_list, dev_label_list = CalACC(erc_model, dev_dataloader)
dev_pre, dev_rec, dev_fbeta, _ = precision_recall_fscore_support(dev_label_list, dev_pred_list, average='weighted')

NameError: ignored

In [ ]:
from dataset import data_loader
from torch.utils.data import DataLoader

train_dataset = data_loader('./MELD/data/MELD/train_sent_emo.csv')
dev_dataset = data_loader('./MELD/data/MELD/dev_sent_emo.csv')
test_dataset = data_loader('./MELD/data/MELD/test_sent_emo.csv')

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4, collate_fn=train_dataset.collate_fn)
dev_dataloader = DataLoader(dev_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=dev_dataset.collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=test_dataset.collate_fn)

In [ ]:
len(train_dataloader), len(dev_dataloader), len(test_dataloader)

In [ ]:
from model import ERC_model
clsNum = len(train_dataset.emoList)
erc_model = ERC_model(clsNum).cuda()

In [ ]:
## GPU 작동 모델 수정
from transformers import RobertaModel
import torch
import torch.nn as nn

class ERC_model(nn.Module):
    def __init__(self, clsNum):
        super(ERC_model, self).__init__()
        self.com_model = RobertaModel.from_pretrained('roberta-base')
        self.pm_model = RobertaModel.from_pretrained('roberta-base')
        
        """ GRU 세팅 """
        self.hiddenDim = self.com_model.config.hidden_size
        zero = torch.empty(2, 1, self.hiddenDim)
        self.h0 = torch.zeros_like(zero).cuda() # (num_layers * num_directions, batch, hidden_size)
        self.speakerGRU = nn.GRU(self.hiddenDim, self.hiddenDim, 2, dropout=0.3) # (input, hidden, num_layer) (BERT_emb, BERT_emb, num_layer)
        
        """ score matrix """
        self.W = nn.Linear(self.hiddenDim, clsNum)
    def forward(self, batch_padding_token, batch_padding_attention_mask, batch_PM_input):
        """ for CoM """
        batch_com_out = self.com_model(input_ids=batch_padding_token, attention_mask=batch_padding_attention_mask)['last_hidden_state']
        batch_com_final = batch_com_out[:,0,:]
        
        """ GRU 통과 --> PM 결과 """
        batch_pm_gru_final = []
        for PM_inputs in batch_PM_input:
            if PM_inputs:
                pm_outs = []
                for PM_input in PM_inputs:
                    pm_out = self.pm_model(PM_input)['last_hidden_state'][:,0,:]
                    pm_outs.append(pm_out)
                pm_outs = torch.cat(pm_outs, 0).unsqueeze(1) # (speaker_num, batch=1, hidden_dim)
                pm_gru_outs, _ = self.speakerGRU(pm_outs, self.h0) # (speaker_num, batch=1, hidden_dim)
                pm_gru_final = pm_gru_outs[-1,:,:] # (1, hidden_dim)
                batch_pm_gru_final.append(pm_gru_final)
            else:
                batch_pm_gru_final.append(torch.zeros(1, self.hiddenDim).cuda())
        batch_pm_gru_final = torch.cat(batch_pm_gru_final, 0)        
        
        """ score matrix """
        final_output = self.W(batch_com_final + batch_pm_gru_final) # (B, C)
        
        return final_output

In [ ]:
import torch.nn as nn
def CELoss(pred_outs, labels):
    """
        pred_outs: [batch, clsNum]
        labels: [batch]
    """
    loss = nn.CrossEntropyLoss()
    loss_val = loss(pred_outs, labels)
    return loss_val

In [ ]:
import torch
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import pdb

training_epochs = 5
max_grad_norm = 10
lr = 1e-6
num_training_steps = len(train_dataset)*training_epochs
num_warmup_steps = len(train_dataset)
optimizer = torch.optim.AdamW(erc_model.parameters(), lr=lr) # , eps=1e-06, weight_decay=0.01
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

for epoch in tqdm(range(training_epochs)):
    erc_model.train() 
    for i_batch, data in enumerate(train_dataloader):
        batch_padding_token, batch_padding_attention_mask, batch_PM_input, batch_label = data
        batch_padding_token = batch_padding_token.cuda()
        batch_padding_attention_mask = batch_padding_attention_mask.cuda()
        batch_PM_input = [[x2.cuda() for x2 in x1] for x1 in batch_PM_input]
        batch_label = batch_label.cuda()        
        
        """Prediction"""
        pred_logits = erc_model(batch_padding_token, batch_padding_attention_mask, batch_PM_input)
        
        """Loss calculation & training"""
        loss_val = CELoss(pred_logits, batch_label)
        
        loss_val.backward()
        torch.nn.utils.clip_grad_norm_(erc_model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()        
        break
    break

# 모델 저장하기

In [ ]:
import os
def SaveModel(model, path):
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(model.state_dict(), os.path.join(path, 'model.bin'))

In [ ]:
import math
a = -(0.8*math.log(0.8)+0.2*math.log(0.2))
b = -(0.8*math.log(0.5)+0.2*math.log(0.5))
a, b

# 최종 학습 코드

In [ ]:
import torch
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

training_epochs = 5
max_grad_norm = 10
lr = 1e-6
num_training_steps = len(train_dataset)*training_epochs
num_warmup_steps = len(train_dataset)
optimizer = torch.optim.AdamW(erc_model.parameters(), lr=lr) # , eps=1e-06, weight_decay=0.01
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

best_dev_fscore = 0
save_path = '.'
for epoch in tqdm(range(training_epochs)):
    erc_model.train() 
    for i_batch, data in enumerate(tqdm(train_dataloader)):
        batch_padding_token, batch_padding_attention_mask, batch_PM_input, batch_label = data
        batch_padding_token = batch_padding_token.cuda()
        batch_padding_attention_mask = batch_padding_attention_mask.cuda()
        batch_PM_input = [[x2.cuda() for x2 in x1] for x1 in batch_PM_input]
        batch_label = batch_label.cuda()        
        
        """Prediction"""
        pred_logits = erc_model(batch_padding_token, batch_padding_attention_mask, batch_PM_input)
        
        """Loss calculation & training"""
        loss_val = CELoss(pred_logits, batch_label)
        
        loss_val.backward()
        torch.nn.utils.clip_grad_norm_(erc_model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    
    """Dev & Test evaluation"""
    erc_model.eval()
    
    dev_acc, dev_pred_list, dev_label_list = CalACC(erc_model, dev_dataloader)
    dev_pre, dev_rec, dev_fbeta, _ = precision_recall_fscore_support(dev_label_list, dev_pred_list, average='weighted')
    
    print("Dev W-avg F1: {}".format(dev_fbeta))

    """Best Score & Model Save"""
    if dev_fbeta > best_dev_fscore:
        best_dev_fscore = dev_fbeta

        test_acc, test_pred_list, test_label_list = CalACC(erc_model, test_dataloader)
        test_pre, test_rec, test_fbeta, _ = precision_recall_fscore_support(test_label_list, test_pred_list, average='weighted')                

        SaveModel(erc_model, save_path)
        print("Test W-avg F1: {}".format(test_fbeta))

# 파일로 저장하기

In [ ]:
import torch
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

import os
from sklearn.metrics import precision_recall_fscore_support
import torch.nn as nn
import pdb

import logging

# 로그 생성
logger = logging.getLogger()

# 로그의 출력 기준 설정
logger.setLevel(logging.INFO)

# log 출력
stream_handler = logging.StreamHandler()
logger.addHandler(stream_handler)

# log를 파일에 출력
file_handler = logging.FileHandler('erc.log')
logger.addHandler(file_handler)

def CELoss(pred_outs, labels):
    """
        pred_outs: [batch, clsNum]
        labels: [batch]
    """
    loss = nn.CrossEntropyLoss()
    loss_val = loss(pred_outs, labels)
    return loss_val

def SaveModel(model, path):
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(model.state_dict(), os.path.join(path, 'model.bin'))
    
def CalACC(model, dataloader):
    model.eval()
    correct = 0
    label_list = []
    pred_list = []
    
    # label arragne
    with torch.no_grad():
        for i_batch, data in enumerate(tqdm(dataloader)):
            """Prediction"""
            batch_padding_token, batch_padding_attention_mask, batch_PM_input, batch_label = data
            batch_padding_token = batch_padding_token.cuda()
            batch_padding_attention_mask = batch_padding_attention_mask.cuda()
            batch_PM_input = [[x2.cuda() for x2 in x1] for x1 in batch_PM_input]
            batch_label = batch_label.cuda()        

            """Prediction"""
            pred_logits = erc_model(batch_padding_token, batch_padding_attention_mask, batch_PM_input)
            
            """Calculation"""    
            pred_label = pred_logits.argmax(1).item()
            true_label = batch_label.item()
            
            pred_list.append(pred_label)
            label_list.append(true_label)
            if pred_label == true_label:
                correct += 1
        acc = correct/len(dataloader)
    return acc, pred_list, label_list

from dataset import data_loader
from torch.utils.data import DataLoader

train_dataset = data_loader('./MELD/data/MELD/train_sent_emo.csv')
dev_dataset = data_loader('./MELD/data/MELD/dev_sent_emo.csv')
test_dataset = data_loader('./MELD/data/MELD/test_sent_emo.csv')

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4, collate_fn=train_dataset.collate_fn)
dev_dataloader = DataLoader(dev_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=dev_dataset.collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=test_dataset.collate_fn)

from model import ERC_model
clsNum = len(train_dataset.emoList)
erc_model = ERC_model(clsNum).cuda()

""" 하이퍼 파라미터들 """
training_epochs = 10
max_grad_norm = 10
lr = 1e-6
num_training_steps = len(train_dataset)*training_epochs
num_warmup_steps = len(train_dataset)
optimizer = torch.optim.AdamW(erc_model.parameters(), lr=lr) # , eps=1e-06, weight_decay=0.01
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

logger.info("############학습 시작############")
best_dev_fscore = 0
save_path = '.'
for epoch in tqdm(range(training_epochs)):
    erc_model.train() 
    for i_batch, data in enumerate(tqdm(train_dataloader)):
        batch_padding_token, batch_padding_attention_mask, batch_PM_input, batch_label = data
        batch_padding_token = batch_padding_token.cuda()
        batch_padding_attention_mask = batch_padding_attention_mask.cuda()
        batch_PM_input = [[x2.cuda() for x2 in x1] for x1 in batch_PM_input]
        batch_label = batch_label.cuda()        
        
        """Prediction"""
        pred_logits = erc_model(batch_padding_token, batch_padding_attention_mask, batch_PM_input)
        
        """Loss calculation & training"""
        loss_val = CELoss(pred_logits, batch_label)
        
        loss_val.backward()
        torch.nn.utils.clip_grad_norm_(erc_model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    
    """Dev & Test evaluation"""
    erc_model.eval()
    
    dev_acc, dev_pred_list, dev_label_list = CalACC(erc_model, dev_dataloader)
    dev_pre, dev_rec, dev_fbeta, _ = precision_recall_fscore_support(dev_label_list, dev_pred_list, average='weighted')
    
    logger.info("Dev W-avg F1: {}".format(dev_fbeta))
    
    test_acc, test_pred_list, test_label_list = CalACC(erc_model, test_dataloader)
    """Best Score & Model Save"""
    if dev_fbeta > best_dev_fscore:
        best_dev_fscore = dev_fbeta

        test_acc, test_pred_list, test_label_list = CalACC(erc_model, test_dataloader)
        test_pre, test_rec, test_fbeta, _ = precision_recall_fscore_support(test_label_list, test_pred_list, average='weighted')                

        SaveModel(erc_model, save_path)
        logger.info("Epoch:{}, Test W-avg F1: {}".format(epoch, test_fbeta))

In [ ]:
!touch train.py
# 복사하기

In [ ]:
!python3 train.py